In [18]:
import os
import numpy as np
import scipy.io
from PIL import Image

# Define your directories
hsi_mat_dir = r"C:\Harshi\ECS-II\Dataset\HSI_mat"  # Directory with .mat files
tiff_dir = r"C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF"  # Directory with TIFF files
rgb_dir = r"C:\Harshi\ECS-II\Dataset\RGB_7_files"  # Directory with RGB images

# Load HSI data from .mat files
hsi_images = {}
band_wavelengths = {}  # To store the band wavelengths for each HSI image

# Iterate through each .mat file in the HSI directory
for mat_file in os.listdir(hsi_mat_dir):
    if mat_file.endswith('.mat'):
        mat_path = os.path.join(hsi_mat_dir, mat_file)
        mat_data = scipy.io.loadmat(mat_path)

        # Assuming 'cube' holds the hyperspectral data and 'bands' holds the wavelengths
        hsi_cube = mat_data['cube']  # Hyperspectral data (3D array)
        wavelengths = mat_data['bands'].flatten()  # Wavelengths (1D array)
        
        # Store the HSI data
        hsi_images[mat_file] = hsi_cube
        band_wavelengths[mat_file] = wavelengths

# Load TIFF files for additional HSI data
tiff_images = {}
for folder in os.listdir(tiff_dir):
    folder_path = os.path.join(tiff_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Loading folder: {folder_path}")  # Debug print
        tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tiff')]
        for tiff_file in tiff_files:
            img = Image.open(os.path.join(folder_path, tiff_file))
            tiff_images[tiff_file] = np.array(img)  # Store TIFF data

# Load RGB images from the RGB directory
rgb_images = {}
for rgb_file in os.listdir(rgb_dir):
    if rgb_file.endswith('.tif') or rgb_file.endswith('.png'):  # Assuming RGB files are .tif or .png
        img = Image.open(os.path.join(rgb_dir, rgb_file))
        rgb_images[rgb_file] = np.array(img)

# Map the HSI RGB to the corresponding RGB images
mapped_rgb_images = {}
rgb_wavelengths = {
    "red": 650,
    "green": 550,
    "blue": 450
}

for mat_file, hsi_cube in hsi_images.items():
    wavelengths = band_wavelengths[mat_file]

    # Find indices of the closest wavelengths for RGB
    red_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["red"]))
    green_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["green"]))
    blue_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["blue"]))

    # Extract RGB bands from the hyperspectral cube
    rgb_from_hsi = hsi_cube[:, :, [red_band_index, green_band_index, blue_band_index]]

    # Create an RGB image from the extracted bands
    hsi_rgb_image = Image.fromarray(np.clip(rgb_from_hsi, 0, 255).astype(np.uint8))

    # Store the mapped image if a corresponding RGB file exists
    for rgb_file in rgb_images.keys():
        if mat_file.startswith(rgb_file):  # Adjust condition based on how you want to match files
            # Optionally resize if dimensions do not match
            rgb_image = Image.fromarray(rgb_images[rgb_file])
            if hsi_rgb_image.size != rgb_image.size:
                hsi_rgb_image = hsi_rgb_image.resize(rgb_image.size, Image.ANTIALIAS)
            
            mapped_rgb_images[rgb_file] = np.array(hsi_rgb_image)

# Final output check
print("Mapped RGB images (from HSI):", mapped_rgb_images)  # Check the contents of the mapped_rgb_images dictionary
print("Loaded RGB images:", rgb_images)  # Check the contents of the rgb_images dictionary
print("Loaded TIFF images:", tiff_images)  # Check the contents of the tiff_images dictionary

Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0151
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0152
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0153
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0155
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0160
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0161
Loading folder: C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF\ARAD_HS_0163
Mapped RGB images (from HSI): {}
Loaded RGB images: {'ARAD_HS_0151_clean.png': array([[[ 59,  64,  77],
        [ 55,  59,  71],
        [ 51,  55,  68],
        ...,
        [ 50,  54,  64],
        [ 50,  54,  65],
        [ 50,  54,  66]],

       [[ 60,  65,  77],
        [ 57,  61,  73],
        [ 52,  56,  69],
        ...,
        [ 51,  55,  66],
        [ 51,  56,  68],
        [ 52,  56,  69]],

       [[ 59,  63,  76],
        [ 57,  61,  75],
        [ 54,  58,  73],
        ...,
      

In [25]:
import os
import numpy as np
import scipy.io
from PIL import Image

def load_hsi_images(hsi_mat_dir):
    hsi_images = {}
    band_wavelengths = {}
    
    for mat_file in os.listdir(hsi_mat_dir):
        if mat_file.endswith('.mat'):
            mat_path = os.path.join(hsi_mat_dir, mat_file)
            mat_data = scipy.io.loadmat(mat_path)

            hsi_cube = mat_data['cube']
            wavelengths = mat_data['bands'].flatten()
            
            hsi_images[mat_file] = hsi_cube
            band_wavelengths[mat_file] = wavelengths
            
    return hsi_images, band_wavelengths

def load_tiff_images(tiff_dir):
    tiff_images = {}
    for folder in os.listdir(tiff_dir):
        folder_path = os.path.join(tiff_dir, folder)
        if os.path.isdir(folder_path):
            tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tiff')]
            for tiff_file in tiff_files:
                img = Image.open(os.path.join(folder_path, tiff_file))
                tiff_images[tiff_file] = np.array(img)
                
    return tiff_images

def load_rgb_images(rgb_dir):
    rgb_images = {}
    for rgb_file in os.listdir(rgb_dir):
        if rgb_file.endswith('.tif') or rgb_file.endswith('.png'):
            img = Image.open(os.path.join(rgb_dir, rgb_file))
            rgb_images[rgb_file] = np.array(img)
    
    return rgb_images

def map_hsi_to_rgb(hsi_images, band_wavelengths, rgb_images):
    mapped_rgb_images = {}
    rgb_wavelengths = {
        "red": 650,
        "green": 550,
        "blue": 450
    }

    for mat_file, hsi_cube in hsi_images.items():
        wavelengths = band_wavelengths[mat_file]

        # Find indices of the closest wavelengths for RGB
        red_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["red"]))
        green_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["green"]))
        blue_band_index = np.argmin(np.abs(wavelengths - rgb_wavelengths["blue"]))

        # Print the indices and corresponding wavelengths for debugging
        print(f"Processing {mat_file}:")
        print(f"Red band index: {red_band_index}, Wavelength: {wavelengths[red_band_index]}")
        print(f"Green band index: {green_band_index}, Wavelength: {wavelengths[green_band_index]}")
        print(f"Blue band index: {blue_band_index}, Wavelength: {wavelengths[blue_band_index]}")

        # Extract RGB bands from the hyperspectral cube
        rgb_from_hsi = hsi_cube[:, :, [red_band_index, green_band_index, blue_band_index]]

        # Print the min and max values to check if they contain valid data
        print(f"RGB from HSI min: {rgb_from_hsi.min()}, max: {rgb_from_hsi.max()}")

        # Scale the RGB values to the range [0, 255]
        rgb_scaled = np.clip(rgb_from_hsi * 255, 0, 255).astype(np.uint8)

        # Convert to RGB image
        hsi_rgb_image = Image.fromarray(rgb_scaled)

        # Debugging: Check if matching RGB file exists
        base_name = mat_file.split('.')[0]
        rgb_file_name = f"{base_name}_clean.png"

        if rgb_file_name in rgb_images:
            rgb_image = Image.fromarray(rgb_images[rgb_file_name])
            # Resize if dimensions do not match
            if hsi_rgb_image.size != rgb_image.size:
                hsi_rgb_image = hsi_rgb_image.resize(rgb_image.size, Image.ANTIALIAS)

            mapped_rgb_images[rgb_file_name] = np.array(hsi_rgb_image)

    return mapped_rgb_images

def save_mapped_images(mapped_rgb_images, output_directory):
    for rgb_file_name, rgb_image in mapped_rgb_images.items():
        output_path = os.path.join(output_directory, rgb_file_name)
        # Convert to PIL image and save
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(output_path)
        print(f"Saved: {output_path}")

def main():
    hsi_mat_dir = r"C:\Harshi\ECS-II\Dataset\HSI_mat"
    tiff_dir = r"C:\Harshi\ECS-II\Dataset\HSI_Dataset_TIFF"
    rgb_dir = r"C:\Harshi\ECS-II\Dataset\RGB_7_files"

    # Load images
    hsi_images, band_wavelengths = load_hsi_images(hsi_mat_dir)
    tiff_images = load_tiff_images(tiff_dir)
    rgb_images = load_rgb_images(rgb_dir)

    # Map HSI to RGB
    mapped_rgb_images = map_hsi_to_rgb(hsi_images, band_wavelengths, rgb_images)

    # Specify the output directory for the saved images
    output_directory = r"C:\Harshi\ECS-II\Dataset\mapped_imgs"  # Change to your desired output directory

    # Call the function to save mapped RGB images
    save_mapped_images(mapped_rgb_images, output_directory)

    # Final output check
    print("Mapped RGB images (from HSI):", mapped_rgb_images)
    print("Loaded RGB images:", rgb_images)
    print("Loaded TIFF images:", tiff_images)

if __name__ == "__main__":
    main()


Processing ARAD_HS_0151.mat:
Red band index: 25, Wavelength: 650
Green band index: 15, Wavelength: 550
Blue band index: 5, Wavelength: 450
RGB from HSI min: 0.0037179330906203806, max: 0.984675260993368
Processing ARAD_HS_0152.mat:
Red band index: 25, Wavelength: 650
Green band index: 15, Wavelength: 550
Blue band index: 5, Wavelength: 450
RGB from HSI min: 0.003370300657193707, max: 1.0
Processing ARAD_HS_0153.mat:
Red band index: 25, Wavelength: 650
Green band index: 15, Wavelength: 550
Blue band index: 5, Wavelength: 450
RGB from HSI min: 0.0056895345174251145, max: 0.9792154215409938
Processing ARAD_HS_0155.mat:
Red band index: 25, Wavelength: 650
Green band index: 15, Wavelength: 550
Blue band index: 5, Wavelength: 450
RGB from HSI min: 0.002508725504610803, max: 0.9868500978515217
Processing ARAD_HS_0160.mat:
Red band index: 25, Wavelength: 650
Green band index: 15, Wavelength: 550
Blue band index: 5, Wavelength: 450
RGB from HSI min: 0.0045684755646904305, max: 0.993193640151656